<a href="https://colab.research.google.com/github/dishitk/machine-learning/blob/master/Machine_Learning_Project_3_Human_Activity_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [15]:
df = pd.read_csv("/content/drive/MyDrive/train_human_activity.csv")
df.head(10)

,rn,activity,tBodyAcc.mean.X,tBodyAcc.mean.Y,tBodyAcc.mean.Z,tBodyAcc.std.X,tBodyAcc.std.Y,tBodyAcc.std.Z,tBodyAcc.mad.X,tBodyAcc.mad.Y,tBodyAcc.mad.Z,tBodyAcc.max.X,tBodyAcc.max.Y,tBodyAcc.max.Z,tBodyAcc.min.X,tBodyAcc.min.Y,tBodyAcc.min.Z,tBodyAcc.sma,tBodyAcc.energy.X,tBodyAcc.energy.Y,tBodyAcc.energy.Z,tBodyAcc.iqr.X,tBodyAcc.iqr.Y,tBodyAcc.iqr.Z,tBodyAcc.entropy.X,tBodyAcc.entropy.Y,tBodyAcc.entropy.Z,tBodyAcc.arCoeff.X.1,tBodyAcc.arCoeff.X.2,tBodyAcc.arCoeff.X.3,tBodyAcc.arCoeff.X.4,tBodyAcc.arCoeff.Y.1,tBodyAcc.arCoeff.Y.2,tBodyAcc.arCoeff.Y.3,tBodyAcc.arCoeff.Y.4,tBodyAcc.arCoeff.Z.1,tBodyAcc.arCoeff.Z.2,tBodyAcc.arCoeff.Z.3,tBodyAcc.arCoeff.Z.4,tBodyAcc.correlation.X.Y,...,fBodyBodyAccJerkMag.energy,fBodyBodyAccJerkMag.iqr,fBodyBodyAccJerkMag.entropy,fBodyBodyAccJerkMag.maxInds,fBodyBodyAccJerkMag.meanFreq,fBodyBodyAccJerkMag.skewness,fBodyBodyAccJerkMag.kurtosis,fBodyBodyGyroMag.mean,fBodyBodyGyroMag.std,fBodyBodyGyroMag.mad,fBodyBodyGyroMag.max,fBodyBodyGyroMag.min,fBodyBodyGyroMag.sma,fBodyBodyGyroMag.energy,fBodyBodyGyroMag.iqr,fBodyBodyGyroMag.entropy,fBodyBodyGyroMag.maxInds,fBodyBodyGyroMag.meanFreq,fBodyBodyGyroMag.skewness,fBodyBodyGyroMag.kurtosis,fBodyBodyGyroJerkMag.mean,fBodyBodyGyroJerkMag.std,fBodyBodyGyroJerkMag.mad,fBodyBodyGyroJerkMag.max,fBodyBodyGyroJerkMag.min,fBodyBodyGyroJerkMag.sma,fBodyBodyGyroJerkMag.energy,fBodyBodyGyroJerkMag.iqr,fBodyBodyGyroJerkMag.entropy,fBodyBodyGyroJerkMag.maxInds,fBodyBodyGyroJerkMag.meanFreq,fBodyBodyGyroJerkMag.skewness,fBodyBodyGyroJerkMag.kurtosis,angle.tBodyAccMean.gravity,angle.tBodyAccJerkMean.gravityMean,angle.tBodyGyroMean.gravityMean,angle.tBodyGyroJerkMean.gravityMean,angle.X.gravityMean,angle.Y.gravityMean,angle.Z.gravityMean
0,7,STANDING,0.2790,-0.0196,-0.1100,-0.997,-0.967,-0.983,-0.997,-0.966,-0.983,-0.941,-0.566,-0.817,0.851,0.674,0.834,-0.987,-1.000,-1.000,-1.000,-0.997,-0.972,-0.983,-0.637,-0.5150,-0.537,0.360,-0.2330,0.2260,0.0695,0.0643,-0.0764,0.1380,-0.0368,0.231,-0.1150,0.3190,-0.48800,-0.0959,...,-1.000,-0.991,-1.000,-0.905,0.290,-0.669,-0.933,-0.988,-0.985,-0.984,-0.985,-0.995,-0.988,-1.000,-0.989,-0.736,-1.000,-0.2570,-0.32200,-0.6580,-0.996,-0.995,-0.994,-0.994,-0.997,-0.996,-1.000,-0.994,-0.956,-1.000,0.146,-0.217,-0.5640,-0.21300,-0.2310,0.01460,-0.19000,-0.852,0.1820,-0.0430
1,11,STANDING,0.2770,-0.0127,-0.1030,-0.995,-0.973,-0.985,-0.996,-0.974,-0.985,-0.940,-0.555,-0.816,0.845,0.685,0.838,-0.988,-1.000,-1.000,-1.000,-0.996,-0.980,-0.983,-0.629,-0.3970,-0.424,0.346,-0.1740,0.1780,0.1690,-0.0202,0.1360,-0.1140,0.2430,0.301,-0.0764,0.0820,0.15300,0.0085,...,-1.000,-0.984,-1.000,-1.000,0.502,-0.517,-0.786,-0.983,-0.981,-0.982,-0.979,-0.987,-0.983,-1.000,-0.981,-0.641,-1.000,-0.0472,0.00787,-0.2840,-0.991,-0.989,-0.991,-0.987,-0.980,-0.991,-1.000,-0.992,-0.923,-1.000,0.121,0.349,0.0577,0.08070,0.5960,-0.47600,0.11600,-0.852,0.1880,-0.0347
2,14,STANDING,0.2770,-0.0147,-0.1070,-0.999,-0.991,-0.993,-0.999,-0.991,-0.992,-0.943,-0.569,-0.823,0.852,0.693,0.846,-0.997,-1.000,-1.000,-1.000,-0.998,-0.993,-0.990,-0.888,-0.5540,-0.551,0.412,-0.1280,0.2650,0.0677,0.2890,-0.2410,0.3150,-0.1520,0.181,0.0504,-0.0398,-0.00999,0.0436,...,-1.000,-0.994,-1.000,-0.905,0.530,-0.606,-0.869,-0.999,-0.998,-0.998,-0.999,-0.998,-0.999,-1.000,-0.996,-1.000,-0.949,0.3250,-0.75900,-0.9420,-0.999,-0.999,-0.999,-0.999,-0.995,-0.999,-1.000,-0.999,-1.000,-0.968,0.740,-0.564,-0.7660,0.10600,-0.0903,-0.13200,0.49900,-0.850,0.1890,-0.0351
3,15,STANDING,0.2980,0.0271,-0.0617,-0.989,-0.817,-0.902,-0.989,-0.794,-0.888,-0.926,-0.448,-0.731,0.849,0.681,0.838,-0.904,-1.000,-0.989,-0.991,-0.989,-0.770,-0.869,-0.163,0.0546,0.073,0.158,-0.1600,0.1900,-0.3220,-0.3150,0.2790,-0.3070,0.3430,-0.083,0.0273,-0.0465,-0.08650,0.8220,...,-1.000,-0.984,-1.000,-1.000,0.169,-0.522,-0.818,-0.911,-0.884,-0.898,-0.880,-0.847,-0.911,-0.993,-0.956,-0.387,-1.000,-0.1730,0.07620,-0.3040,-0.990,-0.989,-0.990,-0.987,-0.999,-0.990,-1.000,-0.992,-0.923,-1.000,0.131,0.208,-0.0681,0.06230,-0.0587,0.03120,-0.2

In [16]:
Y = df['activity'].to_numpy().copy()
Y

array(['STANDING', 'STANDING', 'STANDING', ..., 'WALKING_UPSTAIRS',
       'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'], dtype=object)

In [17]:
for y in range(len(Y)):
  if Y[y] == 'STANDING' or Y[y] == 'LAYING' or Y[y] == 'SITTING':
    Y[y] = 0
  else:
    Y[y] = 1

In [18]:
df.drop(labels=['rn', 'activity'], axis='columns', inplace=True)
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

In [19]:
neighbors = np.array(range(1, 10))
error = []
for k in neighbors:
    knn = KMeans(n_clusters = k)
    knn.fit(scaled_df)
    error.append(knn.inertia_)
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=neighbors, y=error))
fig.update_layout(title_text='Clusters v/s Inertia', xaxis_title="Number of Clusters",
    yaxis_title="Inertia")
fig.show()

In [20]:
k_means = KMeans(2)
k_means.fit(scaled_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [21]:
print(k_means.labels_)
print(k_means.cluster_centers_)

[0 0 0 ... 1 1 1]
[[-0.00817991  0.06174787  0.05731438 ...  0.36968349 -0.45764643
  -0.4050894 ]
 [ 0.00989804 -0.07471756 -0.06935285 ... -0.44733287  0.5537718
   0.49017554]]


In [22]:
from sklearn.metrics.cluster import homogeneity_score as hs
from sklearn.metrics.cluster import completeness_score as cs, silhouette_score as ss
from sklearn.metrics.cluster import v_measure_score as vms

predictions = k_means.predict(scaled_df)
print("Homogeneity Score: "+str(hs(Y,predictions)))
print("Completeness Score: "+str(cs(Y,predictions)))
print("Silhouette Score: "+str(ss(scaled_df, predictions, metric='euclidean')))
print("V Measure Score: "+str(vms(Y,predictions)))
print("Inertia: "+str(k_means.inertia_))

Homogeneity Score: 0.977400570620251
Completeness Score: 0.977696977792435
Silhouette Score: 0.38990168994287766
V Measure Score: 0.977548751737589
Inertia: 1156484.008987799


In [23]:
from sklearn.decomposition import PCA 
  
p = PCA(n_components= 2) 
train_df = p.fit_transform(scaled_df) 
k_means.fit(train_df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [24]:
p = PCA(random_state = 3)
p.fit(df)
feature = np.array(range(p.n_components_))

fig = go.Figure([go.Bar(x=feature[:11], y=p.explained_variance_[:11])])
fig.update_layout(title_text='Feature v/s Variance', xaxis_title="Features",
    yaxis_title="Variance")
fig.show()

In [25]:
def pca(number_components):
  pca_model = PCA(n_components = number_components, random_state = 3)
  pca_scaled_df = pca_model.fit_transform(scaled_df)
  k_means = KMeans(2)
  k_means.fit(pca_scaled_df)
  
  return k_means.predict(pca_scaled_df)

In [26]:
predictions = pca(2)
print("Homogeneity Score: "+str(hs(Y,predictions)))
print("Completeness Score: "+str(cs(Y,predictions)))
print("Silhouette Score: "+str(ss(scaled_df, predictions, metric='euclidean')))
print("V Measure Score: "+str(vms(Y,predictions)))
print("Inertia: "+str(k_means.inertia_))

Homogeneity Score: 0.9747915997701333
Completeness Score: 0.9750126369261398
Silhouette Score: 0.3898957946984315
V Measure Score: 0.9749021058193341
Inertia: 295753.5444611843
